In [ ]:
%pip install ultralytics shapely opencv-python numpy Pillow


In [123]:
%pip install dill


   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------- ----------------------------- 30.7/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [124]:
import os
from ultralytics import YOLO
from shapely.geometry import Polygon
import cv2
import numpy as np
from PIL import Image
import time
from random import randint
from pathlib import Path
import math

In [125]:
DATA_DIR = "/content/gdrive/My Drive/dataset"
CONFIG_PATH = "/content/gdrive/My Drive/config.yaml"

image_dir = 'dataset/images/val'
mask_image_dir = 'dataset/masks/val'

In [220]:
model_path = "viginet_aug_v9_3_270/segment/train/weights/best.pt"

In [221]:
model = YOLO(model_path)

In [222]:
#List all images from directory

def list_dir(path):
  images  = []

  for filename in os.listdir(path):
    # Check if file extension is an image format
    if filename.endswith(('.jpg', '.jpeg', '.png')):
      # Construct the full image path
      image_path = os.path.join(path, filename).replace("\\", "/")
      # Append the path to the list
      images.append(image_path)

  return images

orig_images  = list_dir(image_dir)
orig_images.sort()

mask_images = list_dir(mask_image_dir)
mask_images.sort()

In [223]:
print(mask_images)
print(orig_images)

['dataset/masks/val/Block_1C1_Row_1_Middle_1023_697b0d723f_21.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1048_7394bb177c_21.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1048_7394bb177c_31.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1063_effef009a7_01.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1063_effef009a7_21.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1068_f7dc5667da_11.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1073_99357aa128_11.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1083_8a16560fb9_31.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1103_92f144ba5f_01.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1113_395ccc992d_01.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1133_d6669beea7_31.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1178_e07c7a45e3_31.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1218_17018811be_21.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1238_61b2da36fe_41.png', 'dataset/masks/val/Block_1C1_Row_1_Middle_1243.png', 'dataset

In [215]:
intersections = []
error_file = []

In [216]:
def calculate_iou(orig_image_path , mask_image_path):
    predicted_output = model.predict(orig_image_path)


    
    for r in predicted_output:
          img = np.copy(r.orig_img)
          img_name = Path(r.path).stem # source image base-name

          # Create binary mask
          b_mask = np.zeros(img.shape[:2], np.uint8)
          

          # Iterate each object contour (multiple detections)
          for ci,c in enumerate(r):
              #  Get detection class name
              label = c.names[c.boxes.cls.tolist().pop()]
              #  Extract contour result
              contour = c.masks.xy.pop()
              #  Changing the type
              contour = contour.astype(np.int32)
              #  Reshaping
              contour = contour.reshape(-1, 1, 2)


              # Draw contour onto mask
              try:
                predict_mask = cv2.drawContours(b_mask,
                                    [contour],
                                    -1,
                                    (255, 255, 255),
                                    cv2.FILLED)
              except:
                  error_file.append(orig_image_path)
                  return -1
              

    mask_img = Image.open(mask_image_path).convert('L')
    mask = np.array(mask_img) > 0
    intersection = np.logical_and(mask, b_mask)
    intersections.append(intersection)
    union = np.logical_or(mask, b_mask)
    iou = np.sum(intersection)/np.sum(union)

    return iou



In [224]:
%%capture
iou = []
for i in range(len(orig_images)):
  current_iou = calculate_iou(orig_images[i] , mask_images[i])
  if(current_iou!=-1 and  (1  - math.isnan(current_iou))):
      iou.append(current_iou)
  else :
      error_file.append(orig_images[i])


image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\dataset\images\val\Block_1C1_Row_1_Middle_1023_697b0d723f_21.png: 320x640 21 grapes, 1244.6ms
Speed: 4.0ms preprocess, 1244.6ms inference, 18.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\dataset\images\val\Block_1C1_Row_1_Middle_1048_7394bb177c_21.png: 320x640 14 grapes, 1138.0ms
Speed: 2.0ms preprocess, 1138.0ms inference, 19.4ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\dataset\images\val\Block_1C1_Row_1_Middle_1048_7394bb177c_31.png: 320x640 13 grapes, 1070.0ms
Speed: 2.0ms preprocess, 1070.0ms inference, 12.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\dataset\images\val\Block_1C1_Row_1_Middle_1063_effef009a7_01.png: 320x640 12 grapes, 1077.3ms


In [225]:
total_sum = sum(iou)
mean = total_sum / len(iou)

print(mean)

0.7179651942095576


In [144]:
print(iou)

[0.7214715880657797, 0.7687699512030488, 0.7340989047461002, 0.6269130964359477, 0.6604726022386422, 0.6366882620007142, 0.6477647392491226, 0.7432045324336368, 0.6025484720965902, 0.530009274389436, 0.7121603933212016, 0.6973236164592814, 0.6657292759706191, 0.6138102917460783, 0.6960056657223797, 0.6720660466088411, 0.6891048154218372, 0.6669678751890237, 0.5865223357219088, 0.689547969906112, 0.6245812510297106, 0.6480243023282175, 0.6506272869837951, 0.7081438487314504, 0.516968127140223, 0.7395736382573239, 0.7646735042107774, 0.7789394216691069, 0.5802768409218663, 0.7254843144606924, 0.628586332811685, 0.7350217214875638, 0.7092642456880868, 0.6696992202005199, 0.6692149731561889, 0.7268801552644347, 0.6694856748360373, 0.6146676406135866, 0.5348059288309591, 0.6315885464161956, 0.590497158363627, 0.4724431292768633, 0.6385492227979275, 0.6375677598175084, 0.6809194703931546, 0.572372060857538, 0.5479392203505025, 0.5633389781504274, 0.6591307066916823, 0.622904435170121, 0.6762

In [219]:
print("dim  " , len(iou))

dim   376


In [98]:
ig = "dataset/images/val/Block_1C1_Row_1_Middle_1023_697b0d723f_21.png"
mask = "dataset/masks/val/Block_1C1_Row_1_Middle_1023_697b0d723f_21.png"

In [227]:
ioutemp = calculate_iou(ig , mask)


image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\dataset\images\val\Block_1C1_Row_1_Middle_1023_697b0d723f_21.png: 320x640 21 grapes, 1051.0ms
Speed: 3.0ms preprocess, 1051.0ms inference, 97.0ms postprocess per image at shape (1, 3, 320, 640)


In [228]:
print(ioutemp)

0.7707217478920295


In [205]:
final_iou={}

In [153]:
model_list = ["viginet_aug_50_epoch/segment/train/weights/best.pt","viginet_aug_100_epoch/segment/train/weights/best.pt" , "viginet_aug_150_epoch/segment/train/weights/best.pt","viginet_aug_200_epoch/segment/train/weights/best.pt","viginet_aug_250_epoch/segment/train/weights/best.pt","viginet_aug_300_epoch/segment/train/weights/best.pt","viginet_aug_350_epoch/segment/train/weights/best.pt"]

In [206]:
model_list9 =["viginet_aug_v9_1/segment/train/weights/best.pt" , "viginet_aug_v9_2_120/segment/train/weights/best.pt","viginet_aug_v9_3_175/segment/train/weights/best.pt","viginet_aug_v9_3_240/segment/train/weights/best.pt"]

In [207]:
model_list = model_list9

In [208]:
def calculate_iou_with_model(orig_image_path , mask_image_path , model):
    predicted_output = model.predict(orig_image_path)


    
    for r in predicted_output:
          img = np.copy(r.orig_img)
          img_name = Path(r.path).stem # source image base-name

          # Create binary mask
          b_mask = np.zeros(img.shape[:2], np.uint8)
          

          # Iterate each object contour (multiple detections)
          for ci,c in enumerate(r):
              #  Get detection class name
              label = c.names[c.boxes.cls.tolist().pop()]
              #  Extract contour result
              contour = c.masks.xy.pop()
              #  Changing the type
              contour = contour.astype(np.int32)
              #  Reshaping
              contour = contour.reshape(-1, 1, 2)


              # Draw contour onto mask
              try:
                predict_mask = cv2.drawContours(b_mask,
                                    [contour],
                                    -1,
                                    (255, 255, 255),
                                    cv2.FILLED)
              except:
                  error_file.append(orig_image_path)
                  return -1
              

    mask_img = Image.open(mask_image_path).convert('L')
    mask = np.array(mask_img) > 0
    intersection = np.logical_and(mask, b_mask)
    intersections.append(intersection)
    union = np.logical_or(mask, b_mask)
    iou = np.sum(intersection)/np.sum(union)

    return iou

In [209]:
%%capture

for i in range(len(model_list)):
    model_path = model_list[i]
    modeltemp = YOLO(model_list[i])

    iou = []
    for i in range(len(orig_images)):
        current_iou = calculate_iou_with_model(orig_images[i] , mask_images[i] , modeltemp)
        if(current_iou!=-1 and  (1  - math.isnan(current_iou))):
            iou.append(current_iou)
        else :
            error_file.append(orig_images[i])

    total_sum = sum(iou)
    mean_iou = total_sum / len(iou)

    # key = model_list[i].split("/")[0]

    final_iou[model_path] = mean_iou
    

    


image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\dataset\images\val\Block_1C1_Row_1_Middle_1023_697b0d723f_21.png: 320x640 21 grapes, 1093.5ms
Speed: 2.0ms preprocess, 1093.5ms inference, 18.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\dataset\images\val\Block_1C1_Row_1_Middle_1048_7394bb177c_21.png: 320x640 16 grapes, 1027.5ms
Speed: 2.0ms preprocess, 1027.5ms inference, 14.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\dataset\images\val\Block_1C1_Row_1_Middle_1048_7394bb177c_31.png: 320x640 16 grapes, 1005.0ms
Speed: 2.0ms preprocess, 1005.0ms inference, 16.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\Kaushik Vishwakarma\OneDrive\Desktop\Projects\DLimageseg-aug\dataset\images\val\Block_1C1_Row_1_Middle_1063_effef009a7_01.png: 320x640 8 grapes, 1158.3ms
S

In [210]:
final_iou

{'viginet_aug_v9_1/segment/train/weights/best.pt': 0.6748043801568386,
 'viginet_aug_v9_2_120/segment/train/weights/best.pt': 0.6963815595991502,
 'viginet_aug_v9_3_175/segment/train/weights/best.pt': 0.7154985849587004,
 'viginet_aug_v9_3_240/segment/train/weights/best.pt': 0.7185244929276607}